<a href="https://colab.research.google.com/github/MargamAbhayRaj/2216183_AIML_Bootcamp_2023/blob/main/sssss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas numpy PyPDF2 transformers sentence-transformers pinecone-client streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from PyPDF2 import PdfReader

def extract_tables_from_pdf(pdf_path):
  reader = PdfReader(pdf_path)
  text = ""
  for page in reader.pages:
    text += page.extract_text()
    return text

# Example
pdf_path = "/content/Sample Financial Statement (1).pdf"
raw_text = extract_tables_from_pdf(pdf_path)

# Preprocessing (convert raw text into structured data)
# This will need fine-tuning based on your specific P&L statement format.
def preprocess_financial_data(raw_text):
    # Assume raw_text is structured into rows/columns
    lines = raw_text.split('\n')
    # Mock parsing example
    data = [line.split() for line in lines if line.strip()]
    df = pd.DataFrame(data)
    return df

financial_data = preprocess_financial_data(raw_text)


In [ ]:
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer

embed_model = SentenceTransformer('all-mpnet-base-v2')

# Create a Pinecone client instance
pinecone_client = Pinecone(
    api_key="pcsk_3hHcWk_JLwSUVKo5hB5cdsScpwSazh1GgT2Z3XNQLCCi7rR3XuztWsxiij8oK1GtxDDrMs"
)

# Define the index name and embedding dimension
index_name = "financial-qabot"
embedding_dim = 768  # Embedding size for 'all-MiniLM-L6-v2' # Keep this consistent with index dimension

# Check if the index exists; if not, create it
# Delete the existing index if it has the wrong dimension
if index_name in pinecone_client.list_indexes().names():
    index_info = pinecone_client.describe_index(index_name)
    if index_info.dimension != embedding_dim:
        print("Deleting existing index with incorrect dimension...")
        pinecone_client.delete_index(index_name)
        print("Index deleted successfully.")

if index_name not in pinecone_client.list_indexes().names():
    print("Creating new index...")
    pinecone_client.create_index(
        name=index_name,
        dimension=embedding_dim,  # Use the correct embedding dimension
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"  # Update to your preferred region
        )
    )
    print("Index created successfully.")
# Access the index
index = pinecone_client.Index(index_name)

# Embed and upload data
def embed_and_store_data(df):
    for idx, row in df.iterrows():
        # Convert all values to strings and handle None/NaN values
        text = " ".join([str(x) for x in row.values if x is not None])  # Moved before embedding calculation
        embedding = embed_model.encode(text).tolist()
        index.upsert([(str(idx), embedding, {"content": text})])
        text = " ".join([str(x) for x in row.values if x is not None])

embed_and_store_data(financial_data)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
!pip install pandas numpy PyPDF2 transformers sentence-transformers pinecone-client streamlit

import pandas as pd
from PyPDF2 import PdfReader
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
from transformers import pipeline

def extract_tables_from_pdf(pdf_path):
  reader = PdfReader(pdf_path)
  text = ""
  for page in reader.pages:
    text += page.extract_text()
    return text

def preprocess_financial_data(raw_text):
    lines = raw_text.split('\n')
    data = [line.split() for line in lines if line.strip()]
    df = pd.DataFrame(data)
    return df

def embed_and_store_data(df):
    for idx, row in df.iterrows():
        # Convert all values to strings and handle None/NaN values
        text = " ".join([str(x) for x in row.values if x is not None])  # Moved before embedding calculation
        embedding = embed_model.encode(text).tolist()
        index.upsert([(str(idx), embedding, {"content": text})])

def get_relevant_context(query):
    query_embedding = embed_model.encode(query).tolist()
    search_results = index.query(vector=query_embedding, top_k=5, include_metadata=True)
    return " ".join([res["metadata"]["content"] for res in search_results["matches"]])

def answer_question(query):
    context = get_relevant_context(query)
    response = qa_pipeline(question=query, context=context)
    return response["answer"]

# --- Main Execution ---
pdf_path = "/content/Sample Financial Statement (1).pdf"
raw_text = extract_tables_from_pdf(pdf_path)
financial_data = preprocess_financial_data(raw_text)

embed_model = SentenceTransformer('all-mpnet-base-v2')
pinecone_client = Pinecone(api_key="pcsk_3hHcWk_JLwSUVKo5hB5cdsScpwSazh1GgT2Z3XNQLCCi7rR3XuztWsxiij8oK1GtxDDrMs") # Replace with your actual API key

index_name = "financial-qabot"
embedding_dim = 768

if index_name in pinecone_client.list_indexes().names():
    index_info = pinecone_client.describe_index(index_name)
    if index_info.dimension != embedding_dim:
        print("Deleting existing index with incorrect dimension...")
        pinecone_client.delete_index(index_name)
        print("Index deleted successfully.")

if index_name not in pinecone_client.list_indexes().names():
    print("Creating new index...")
    pinecone_client.create_index(
        name=index_name,
        dimension=embedding_dim,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print("Index created successfully.")

index = pinecone_client.Index(index_name)
embed_and_store_data(financial_data)

qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

# Test the pipeline
query = "What is the gross profit for Q3 2023?"
print(answer_question(query))

# --- Streamlit App (Optional) ---
import streamlit as st

st.title("Financial QA Bot")
uploaded_file = st.file_uploader("Upload a P&L PDF", type=["pdf"])
query = st.text_input("Ask a financial question:")

if uploaded_file:
    raw_text = extract_tables_from_pdf(uploaded_file)
    financial_data = preprocess_financial_data(raw_text)
    embed_and_store_data(financial_data)

if query:
    answer = answer_question(query)
    st.write(f"Answer: {answer}")

config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


26,233


2025-01-18 10:46:36.854 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 10:46:37.241 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-01-18 10:46:37.245 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 10:46:37.249 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 10:46:37.254 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 10:46:37.256 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 10:46:37.257 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 10:46:37.258 Thread 'MainThread': mi

In [ ]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

def get_relevant_context(query):
    # Search Pinecone for relevant rows
    query_embedding = embed_model.encode(query).tolist()
    # Use keyword arguments for index.query()
    search_results = index.query(vector=query_embedding, top_k=5, include_metadata=True)
    return " ".join([res["metadata"]["content"] for res in search_results["matches"]])
def answer_question(query):
    context = get_relevant_context(query)
    response = qa_pipeline(question=query, context=context)
    return response["answer"]

# Test the pipeline
query = "What is the gross profit for Q3 2023?"
print(answer_question(query))


Device set to use cpu


26,233


In [ ]:
import streamlit as st

st.title("Financial QA Bot")
uploaded_file = st.file_uploader("Upload a P&L PDF", type=["pdf"])
query = st.text_input("Ask a financial question:")

if uploaded_file:
    raw_text = extract_tables_from_pdf(uploaded_file)
    financial_data = preprocess_financial_data(raw_text)
    embed_and_store_data(financial_data)

if query:
    answer = answer_question(query)
    st.write(f"Answer: {answer}")


2025-01-18 10:46:52.907 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 10:46:52.910 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 10:46:52.915 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 10:46:52.920 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 10:46:52.928 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 10:46:52.935 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 10:46:52.937 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 10:46:52.941 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
!pip install streamlit
!pip install pyngrok


In [ ]:
%%writefile app.py

import streamlit as st

st.title("Financial QA Bot")
st.write("Upload a financial statement and ask questions about it!")

#uploaded_file = st.file_uploader("Upload a P&L PDF", type=["pdf"])
'''
query = st.text_input("Ask a financial question:")

if uploaded_file:
    st.write("File uploaded successfully!")
    # You can process the file and implement your logic here.

if query:
    st.write(f"Answer to your query: {query}")  # Placeholder for actual logic
'''
embed_model = SentenceTransformer('all-mpnet-base-v2')
query = "What is the gross profit for Q3 2023?"
query_embedding = embed_model.encode(query).tolist()


search_results = index.query(vector=query_embedding, top_k=5, include_metadata=True)


print("Search Results:")
for match in search_results["matches"]:
    print(match["metadata"]["content"])



Overwriting app.py


In [ ]:
import streamlit as st
from sentence_transformers import SentenceTransformer  # Import this for embeddings


In [ ]:
pip install sentence-transformers


In [ ]:

import streamlit as st
from sentence_transformers import SentenceTransformer  # Import the required class

# Initialize the SentenceTransformer model
embed_model = SentenceTransformer('all-mpnet-base-v2')  # Ensure the model is defined

st.title("Financial QA Bot")
uploaded_file = st.file_uploader("Upload a P&L PDF", type=["pdf"])

query = st.text_input("Ask a financial question:")

if uploaded_file:
    st.write("File uploaded successfully!")
    # You can process the file and implement your logic here.

if query:
    st.write(f"Answer to your query: {query}")  # Placeholder for actual logic


2025-01-18 11:45:22.019 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 11:45:22.020 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 11:45:22.023 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 11:45:22.027 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 11:45:22.029 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 11:45:22.030 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 11:45:22.032 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 11:45:22.034 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
from pyngrok import ngrok
import subprocess

# Set your authtoken
ngrok.set_auth_token("2rkVq0WRwwlhQWfJ5g4Id0J0CLo_29LmeeBuDE1KC3HVpbsoJ")  # Replace with your actual authtoken

# Kill any existing ngrok tunnels
ngrok.kill()

# Run Streamlit in the background
process = subprocess.Popen(['streamlit', 'run', 'app.py', '--server.port', '8501'])

# Expose Streamlit on the web via ngrok
# Use addr instead of port
public_url = ngrok.connect(addr='8501')
print(f"Streamlit app is running on: {public_url}")

Streamlit app is running on: NgrokTunnel: "https://600c-34-138-109-34.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
# Streamlit app is running on: http://<ngrok_url>

In [ ]:

query = input("")
query_embedding = embed_model.encode(query).tolist()


search_results = index.query(vector=query_embedding, top_k=5, include_metadata=True)


print("Search Results:")
for match in search_results["matches"]:
    print(match["metadata"]["content"])


"What is the gross profit for Q3 2024?"
Search Results:
Gross profit 11,175 11,430 46,257 44,414
Revenue from operation for the three months and year ended March 31, 2024 and March 31, 2023 are as follows:
Profit for the period — — — — 24,095 — — — — — — — — 24,095 13 24,108
Profit for the period — — — — 26,233 — — — — — — — — 26,233 15 26,248
Profit for the period 7,975 6,134 26,248 24,108
